# 🧱 06. 패딩 (Padding): 데이터의 키 맞추기

## 1. 개요
문장마다 길이가 다릅니다. 어떤 문장은 3단어, 어떤 문장은 100단어죠.
하지만 컴퓨터는 **직사각형 모양의 행렬(Matrix)**로 데이터를 받아야 계산을 빨리 할 수 있습니다.
그래서 우리는 문장들의 길이를 **똑같이 맞춰주는 작업**을 해야 합니다.

> **쉬운 비유**:  
> - 테트리스 게임을 할 때 빈 공간이 있으면 블록이 안 사라지죠?
> - 빈 공간을 `0`이라는 벽돌로 채워서 네모 반듯하게 만드는 것이 바로 **패딩(Padding)**입니다.

---

## 2. 주요 개념

| 용어 | 설명 | 예시 |
|:---:|---|---|
| **Padding (채우기)** | 길이가 짧으면 `0`을 채워 넣습니다. | `[1, 2]` ➡️ `[0, 0, 1, 2]` |
| **Truncation (자르기)** | 길이가 너무 길면 잘라버립니다. | `[1, 2, 3, 4, 5]` ➡️ `[1, 2, 3]` |

### Pre vs Post (어디를 채울까?)
- **Pre-padding** (앞): `[0, 0, 1, 2]` 
  - **RNN에서 선호!** (중요한 내용이 뒤에 나올수록 기억이 잘 남아서)
- **Post-padding** (뒤): `[1, 2, 0, 0]`
  - 사람 눈에는 더 자연스러움.

In [ ]:
# 예제 데이터: 길이가 제각각인 문장들 (이미 숫자로 변환되었다고 가정)
# 정수 인코딩이 끝난 상태입니다.
encoded_sentences = [
    [1, 2],                 # 길이 2
    [1, 2, 3, 4, 5],        # 길이 5
    [1],                    # 길이 1
    [1, 2, 3, 4, 5, 6, 7]   # 길이 7
]

print("문장 개수:", len(encoded_sentences))
print("각 문장의 길이:", [len(s) for s in encoded_sentences])

## 3. 원리 이해: 패딩 직접 구현하기

가장 긴 문장 길이에 맞춰서 나머지를 `0`으로 채워봅시다.

In [ ]:
# 1. 가장 긴 문장 길이 찾기
max_len = max(len(s) for s in encoded_sentences)
print(f"가장 긴 문장은 {max_len}단어네요. 모두 이 길이로 맞춥니다.")

# 2. 패딩 넣기 (Post-padding 방식: 뒤에 0 채우기)
padded_sentences = []
for sentence in encoded_sentences:
    # 필요한 0의 개수 = (목표 길이) - (현재 길이)
    zero_count = max_len - len(sentence)
    
    # 원본 문장 + 0들
    padded = sentence + [0] * zero_count 
    padded_sentences.append(padded)

print("\n패딩 결과:")
for p in padded_sentences:
    print(p)
    
# 결과: 모두 길이가 7로 맞춰져서 '직사각형'이 되었습니다!

## 4. 실전: Keras `pad_sequences`

직접 짤 필요 없이 Keras 함수를 쓰면 Pre/Post 설정을 쉽게 할 수 있습니다.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Pre-padding (기본값): 앞에 0 채우기
pre_padded = pad_sequences(encoded_sentences)

print("--- Pre-padding (앞에 0) ---")
print(pre_padded)
# 뒷부분(실제 데이터)이 끝에 몰려있어서 RNN이 마지막 기억을 잘 유지합니다.

# 2. Post-padding: 뒤에 0 채우기
post_padded = pad_sequences(encoded_sentences, padding='post')

print("\n--- Post-padding (뒤에 0) ---")
print(post_padded)

### 길이 제한하기 (Truncating)
"나는 1000단어짜리 문장은 필요 없어! 딱 5단어까지만 볼래."
이럴 때 `maxlen`을 설정합니다.

In [ ]:
# 길이를 3으로 제한!
# truncating='pre' (기본값): 앞부분을 자름 (왜? 뒤에 있는 말이 보통 더 중요하니까)
# truncating='post': 뒷부분을 자름

truncated = pad_sequences(encoded_sentences, maxlen=3, truncating='post')

print("--- 길이 3으로 자르기 (뒷부분 삭제) ---")
print(truncated)
# [1, 2, 3, 4, 5] ➡️ [1, 2, 3] (뒤에 4, 5가 잘림)

## 마무리

1. 컴퓨터에게 데이터를 줄 때는 **직사각형(Matrix)** 모양이어야 합니다.
2. 빈 곳은 `0`으로 채웁니다.
3. RNN을 쓴다면 **Pre-padding(앞에 0)**이 유리합니다.
   - 이유: RNN은 시간이 지날수록 앞의 기억이 흐릿해지는데, 0(무의미)을 처리하느라 기억력을 낭비하는 것보다, 마지막에 실제 데이터를 보는 게 낫기 때문입니다.